# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [ ]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [3]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-Astronomy"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [5]:
# Let's test the base model before training
prompt = "Why can't light escape from a black hole?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt = True)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Why can't light escape from a black hole?
assistant
Why can't light escape from a black hole?

## 1.1.1.1

The black hole is a region of space where the gravitational force is so strong that nothing can escape.

## 1.1.1.2

The black hole is a region of space where the gravitational force is so strong that nothing can escape.

## 1.1.1.3

The black hole is a region of space where the gravitational


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [12]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset("David-Xu/astronomy-stack-dpo")


In [13]:
ds['train'][0]

{'prompt': 'How do I calculate the inclination of an object with an amateur telescope?',
 'chosen': [{'content': 'How do I calculate the inclination of an object with an amateur telescope?',
   'role': 'user'},
  {'content': 'Any telescope can be made to give you the information that you are looking for.  The first thing that you will need to know is the location of the ecliptic which varies throughout the year.  Or are you looking to find in relation to the the celestial equator?\n\nhttps://en.wikipedia.org/wiki/Celestial_equator\n\nEither way, you would start of the same by find the Declination (D) and Right Ascension (R.A) of the object in question.  You do not need a telescope to find this, unless you are unable to see the object with the naked eye.  If the telescope has a polar mount and is properly set up.  You can read the R.A. and D off of the mount.  \n\nOtherwise it will require some trigonometry and knowing where you are (at least your latitude)\n\nYou will need to determine

In [17]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

def process_dataset(sample):
  messages = sample['chosen']
  return {'messages': messages}

ds_chatml = ds['train'].map(
    process_dataset,
    remove_columns = ds['train'].column_names
)
ds_chatml[0]

{'messages': [{'content': 'How do I calculate the inclination of an object with an amateur telescope?',
   'role': 'user'},
  {'content': 'Any telescope can be made to give you the information that you are looking for.  The first thing that you will need to know is the location of the ecliptic which varies throughout the year.  Or are you looking to find in relation to the the celestial equator?\n\nhttps://en.wikipedia.org/wiki/Celestial_equator\n\nEither way, you would start of the same by find the Declination (D) and Right Ascension (R.A) of the object in question.  You do not need a telescope to find this, unless you are unable to see the object with the naked eye.  If the telescope has a polar mount and is properly set up.  You can read the R.A. and D off of the mount.  \n\nOtherwise it will require some trigonometry and knowing where you are (at least your latitude)\n\nYou will need to determine the Altitude and Azimuth of the object.  Which is really just the direction from North

In [19]:
ds_chatml = ds_chatml.train_test_split(test_size=0.2, seed = 42)
ds_chatml_train = ds_chatml['train']
ds_chatml_test = ds_chatml['test']

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [24]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    report_to  = None,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds_chatml_train,
    tokenizer=tokenizer,
    eval_dataset=ds_chatml_test,
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/15948 [00:00<?, ? examples/s]

Map:   0%|          | 0/3987 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [22]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,2.464000,2.498072
100,2.488300,2.475387
150,2.538300,2.457941
200,2.389200,2.444991
250,2.524800,2.435325
300,2.400800,2.424933
350,2.435900,2.414716
400,2.479800,2.407331
450,2.433200,2.398982
500,2.404800,2.391631


In [1]:
trainer.push_to_hub(tags=finetune_tags)

NameError: name 'trainer' is not defined

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Why can't light escape from a black hole?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
ft_model =AutoModelForCausalLM.from_pretrained(f"./{finetune_name}").to(device)
outputs = ft_model.generate(
    **inputs,
    max_new_tokens=200,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_k=10,
    temperature=0.8,
)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.